# Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
from crewai import Agent, Task, Crew
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

/Users/akaoui/Desktop/Github/ml-projects/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:898: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


## Role Playing, Focus and Cooperation

In [2]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [4]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

In [5]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [6]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

In [7]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [8]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [9]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [12]:
inputs = {
    "customer": "Datachain.AI",
    "person": "Chihab AKAOUI",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: Datachain.AI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Chihab AKAOUI from Datachain.AI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide Chihab AKAOUI from Datachain.AI with the best support possible regarding setting up a Crew and adding memory to it. I should use the "Read website content" tool to gather information on how to do this properly.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

Introduction - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationGet Sta

/Users/akaoui/Desktop/Github/ml-projects/.venv/lib/python3.12/site-packages/crewai/utilities/evaluators/task_evaluator.py:51: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  converter = Converter(


Final Answer: 

Hello Chihab AKAOUI,

Thank you for reaching out to us at crewAI for assistance with setting up a Crew and adding memory to it. I understand the importance of this task for Datachain.AI, and I am here to provide you with detailed guidance on how to achieve this.

To set up a Crew and add memory to it, you can follow the steps outlined in our documentation on creating a Crew and kicking it off. By creating a Crew in crewAI, you can optimize for autonomy and collaborative intelligence, enabling you to create AI teams where each agent has specific roles, tools, and goals. Here is an overview of the key components involved:

1. Crew: The top-level organization that manages AI agent teams, oversees workflows, ensures collaboration, and delivers outcomes.
2. AI Agents: Specialized team members with specific roles, tools, and the ability to delegate tasks and make autonomous decisions.
3. Process: A workflow management system that defines collaboration patterns, controls task 

/Users/akaoui/Desktop/Github/ml-projects/.venv/lib/python3.12/site-packages/crewai/utilities/evaluators/task_evaluator.py:51: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  converter = Converter(


I now can give a great answer

Final Answer: 
To set up a Crew and add memory to it for Datachain.AI, follow these specific steps:

1. Define the Crew: Start by identifying the members of the Crew and their roles within the team. Make sure to clearly outline the goals and objectives of the Crew to ensure everyone is aligned.

2. Create the Crew: Using the CrewAI platform, navigate to the "Crews" section and click on the "Create New Crew" button. Enter the necessary information such as the Crew name, description, and members. Assign roles and permissions as needed.

3. Configure Memory Management: To add memory to the Crew, consider the memory requirements of the tasks and processes that the Crew will be working on. Efficient memory management is crucial for optimizing performance and preventing memory-related issues. Make sure to allocate the appropriate amount of memory resources to the Crew to support their work effectively.

4. Implement Memory Management: Utilize the memory managem

/Users/akaoui/Desktop/Github/ml-projects/.venv/lib/python3.12/site-packages/crewai/utilities/evaluators/task_evaluator.py:51: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  converter = Converter(


In [13]:
from IPython.display import Markdown
Markdown(result)

Hello Chihab AKAOUI,

Thank you for reaching out to us at crewAI for assistance with setting up a Crew and adding memory to it. I understand the importance of this task for Datachain.AI, and I am here to provide you with detailed guidance on how to achieve this.

To set up a Crew and add memory to it for Datachain.AI, follow these specific steps:

1. Define the Crew: Start by identifying the members of the Crew and their roles within the team. Make sure to clearly outline the goals and objectives of the Crew to ensure everyone is aligned.

2. Create the Crew: Using the CrewAI platform, navigate to the "Crews" section and click on the "Create New Crew" button. Enter the necessary information such as the Crew name, description, and members. Assign roles and permissions as needed.

3. Configure Memory Management: To add memory to the Crew, consider the memory requirements of the tasks and processes that the Crew will be working on. Efficient memory management is crucial for optimizing performance and preventing memory-related issues. Make sure to allocate the appropriate amount of memory resources to the Crew to support their work effectively.

4. Implement Memory Management: Utilize the memory management features available within the CrewAI platform to monitor and control memory usage. Set up alerts and notifications for memory thresholds to proactively address any potential issues. Regularly review and adjust memory allocations based on the Crew's workload and performance requirements.

5. Monitor and Optimize: Continuously monitor the memory usage of the Crew and make adjustments as necessary to ensure optimal performance. Collaborate with the Crew members to gather feedback on memory allocation and make improvements based on their input.

By following these specific steps, you can effectively set up a Crew and add memory to it for Datachain.AI, ensuring smooth workflow management and efficient memory utilization.

If you have any specific questions or need further assistance, please feel free to reach out to us. We are here to help you every step of the way in achieving your goals with crewAI.

Best regards,

[Your Name]
Senior Support Representative
crewAI